<div align=center>
<font face="XB Niloofar">
<font size=4>
گزارش آزمایش نهم
<p></p>
<font size=4>
فرهاد فلاح 97102214 
<p></p>


In [1]:
import cv2
import numpy as np
import os
from sklearn.cluster import KMeans
from tqdm import tqdm
from scipy.cluster.vq import vq
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit


In [2]:
activity_dict = {'handclapping':0 ,'handwaving':1 ,  'running':2}

نرخ های مختلفی برای نمونه برداری امتحان شد در نرخ 0.1 هم دقت هم سرعت مناسب بود

In [43]:
path = 'Dataset'
category = 'handclapping'
percentage = 0.1
features = []
for category in ['handclapping' ,'handwaving',  'running']:
    for file in tqdm(os.listdir(f'{path}/{category}')):
        Mags = []
        Angs = []
        #file = 'person01_handclapping_d2_uncomp.avi'
        cap = cv2.VideoCapture(f'{path}/{category}/{file}')
        l = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))   
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        ret, frame1 = cap.read()
        height = int(h * percentage)
        width = int(w * percentage)
        dim = (width,height)
        frame1 = cv2.resize(frame1, dim, interpolation = cv2.INTER_AREA)
        prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        hsv = np.zeros_like(frame1)
        while(1):
            ret, frame2 = cap.read()
            if not ret:
                break

            frame2 = cv2.resize(frame2, dim, interpolation = cv2.INTER_AREA)
            next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
            flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
            Mags . append(mag.ravel())
            Angs.append(ang.ravel())
            prvs = next
        Mags = np.array(Mags)
        Angs = np.array(Angs)
        feature_vec = np.append(Mags,Angs,axis=1)
       
        features.append(feature_vec)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 12.88it/s]


In [44]:
handclapping = features[0:99]
handwaving = features[99:199]
running = features[199:]

In [45]:
y_train = np.zeros((80,1))
y_train = np.append(y_train,np.ones((80,1)),axis = 0)
y_train = np.append(y_train,2*np.ones((80,1)),axis = 0)
y_train.shape

(240, 1)

In [46]:
train = handclapping[0:80] +  handwaving[0:80] +  running[0:80]

len(train)

240

In [47]:
test = handclapping[80:] +  handwaving[80:] +  running[80:]
y_test = np.zeros((19,1))
y_test = np.append(y_test , np.ones((20,1)),axis = 0)
y_test = np.append(y_test , 2*np.ones((20,1)),axis = 0)

In [48]:
train_features = []
n_clusters = 180
for video in (train):
    features = video
    for feature in features:
        train_features.append(feature) 
    
len(train_features)


107968

In [49]:
kmeans = KMeans(init='k-means++', n_clusters=n_clusters, n_init=2)
kmeans.fit(train_features)

KMeans(n_clusters=180, n_init=2)

In [50]:
clusters = kmeans.cluster_centers_
clusters.shape

(180, 384)

In [55]:
bow =[]
for sample in train:
    labels = vq(sample, clusters)[0]
    hist = np.histogram(labels,bins = np.arange(n_clusters+1))[0]
    bow.append(hist)
bow = np.array(bow)   
bow.shape

(240, 180)

(264, 384)

In [56]:
scaler = StandardScaler()
X_train = bow
scaler.fit(X_train)
X_train = scaler.transform(X_train)



svc = SVC(kernel = 'rbf')

C_range = np.logspace(-10, 10, 21)
gamma_range = np.logspace(-10, 10, 21)
param_grid = dict(gamma=gamma_range, C=C_range)
#cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(svc, param_grid=param_grid)
grid.fit(X_train, y_train.ravel())

print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)

The best parameters are {'C': 10.0, 'gamma': 0.001} with a score of 0.89


In [57]:
C, gamma = grid.best_params_['C'], grid.best_params_['gamma']
clf = SVC(C=C, gamma=gamma)
clf.fit(X_train, y_train.ravel())
y_train_p= clf.predict(X_train)
#y_test_p = clf.predict(X_test)

print('confusion matrix for training set:')
print(confusion_matrix(y_train, y_train_p))
print('accuracy for training set:')
print(accuracy_score(y_train, y_train_p))
#print('\nconfusion matrix for validation set:')
#print(confusion_matrix(y_test, y_test_p))

confusion matrix for training set:
[[79  1  0]
 [ 0 80  0]
 [ 0  0 80]]
accuracy for training set:
0.9958333333333333


In [58]:
bow_test =[]
for sample in test:
    labels = vq(sample, clusters)[0]
    hist = np.histogram(labels,bins = np.arange(n_clusters+1))[0]
    bow_test.append(hist)
bow_test = np.array(bow_test)   
bow_test.shape

(59, 180)

In [59]:
X_test = bow_test
scaler.fit(X_test)
X_test = scaler.transform(X_test)
y_test_p = clf.predict(X_test)
print('\nconfusion matrix for test set:')
print(confusion_matrix(y_test, y_test_p))
print('accuracy for test set:')
print(accuracy_score(y_test, y_test_p))


confusion matrix for test set:
[[16  3  0]
 [ 6 14  0]
 [ 0  0 20]]
accuracy for test set:
0.847457627118644


In [60]:
n_clusters = 200

kmeans2 = KMeans(init='k-means++', n_clusters=n_clusters, n_init=2)
kmeans2.fit(train_features)

clusters2 = kmeans2.cluster_centers_

bow =[]
for sample in train:
    labels = vq(sample, clusters2)[0]
    hist = np.histogram(labels,bins = np.arange(n_clusters+1))[0]
    bow.append(hist)
bow = np.array(bow)   
bow.shape

scaler = StandardScaler()
X_train = bow
scaler.fit(X_train)
X_train = scaler.transform(X_train)



svc = SVC(kernel = 'rbf')

C_range = np.logspace(-10, 10, 21)
gamma_range = np.logspace(-10, 10, 21)
param_grid = dict(gamma=gamma_range, C=C_range)
#cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(svc, param_grid=param_grid)
grid.fit(X_train, y_train.ravel())

print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)

C, gamma = grid.best_params_['C'], grid.best_params_['gamma']
clf2 = SVC(C=C, gamma=gamma)
clf2.fit(X_train, y_train.ravel())
y_train_p= clf2.predict(X_train)

print('confusion matrix for training set:')
print(confusion_matrix(y_train, y_train_p))
print('accuracy for training set:')
print(accuracy_score(y_train, y_train_p))

bow_test =[]
for sample in test:
    labels = vq(sample, clusters2)[0]
    hist = np.histogram(labels,bins = np.arange(n_clusters+1))[0]
    bow_test.append(hist)
bow_test = np.array(bow_test)   

X_test = bow_test
scaler.fit(X_test)
X_test = scaler.transform(X_test)
y_test_p = clf2.predict(X_test)
print('\nconfusion matrix for test set:')
print(confusion_matrix(y_test, y_test_p))
print('accuracy for test set:')
print(accuracy_score(y_test, y_test_p))

The best parameters are {'C': 10000000.0, 'gamma': 1e-09} with a score of 0.90
confusion matrix for training set:
[[77  3  0]
 [ 5 75  0]
 [ 0  0 80]]
accuracy for training set:
0.9666666666666667

confusion matrix for test set:
[[18  1  0]
 [ 5 15  0]
 [ 1  0 19]]
accuracy for test set:
0.8813559322033898


In [61]:
n_clusters = 220

kmeans3 = KMeans(init='k-means++', n_clusters=n_clusters, n_init=2)
kmeans3.fit(train_features)

clusters3 = kmeans3.cluster_centers_


bow =[]
for sample in train:
    labels = vq(sample, clusters3)[0]
    hist = np.histogram(labels,bins = np.arange(n_clusters+1))[0]
    bow.append(hist)
bow = np.array(bow)   
bow.shape

scaler = StandardScaler()
X_train = bow
scaler.fit(X_train)
X_train = scaler.transform(X_train)



svc = SVC(kernel = 'rbf')

C_range = np.logspace(-10, 10, 21)
gamma_range = np.logspace(-10, 10, 21)
param_grid = dict(gamma=gamma_range, C=C_range)
#cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(svc, param_grid=param_grid)
grid.fit(X_train, y_train.ravel())

print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)

C, gamma = grid.best_params_['C'], grid.best_params_['gamma']
clf3 = SVC(C=C, gamma=gamma)
clf3.fit(X_train, y_train.ravel())
y_train_p= clf3.predict(X_train)

print('confusion matrix for training set:')
print(confusion_matrix(y_train, y_train_p))
print('accuracy for training set:')
print(accuracy_score(y_train, y_train_p))

bow_test =[]
for sample in test:
    labels = vq(sample, clusters3)[0]
    hist = np.histogram(labels,bins = np.arange(n_clusters+1))[0]
    bow_test.append(hist)
bow_test = np.array(bow_test)   

X_test = bow_test
scaler.fit(X_test)
X_test = scaler.transform(X_test)
y_test_p = clf3.predict(X_test)
print('\nconfusion matrix for test set:')
print(confusion_matrix(y_test, y_test_p))
print('accuracy for test set:')
print(accuracy_score(y_test, y_test_p))

The best parameters are {'C': 1000000.0, 'gamma': 1e-09} with a score of 0.90
confusion matrix for training set:
[[76  4  0]
 [ 9 71  0]
 [ 0  0 80]]
accuracy for training set:
0.9458333333333333

confusion matrix for test set:
[[19  0  0]
 [ 6 14  0]
 [ 0  0 20]]
accuracy for test set:
0.8983050847457628


همانطور که مشاهده میشود با افزایش بین ها دقت اموزش کم شده(اور فیت کمتر میشود)ولی دقت تست ما بهتر میگردد.

In [71]:
cap = cv2.VideoCapture('test1.mp4')
l = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))   
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(l,w,h)
Mags = []
Angs = []
ret, frame1 = cap.read()
height = int(h * percentage)
width = int(w * percentage)
dim = (16,12)
frame1 = cv2.resize(frame1, dim, interpolation = cv2.INTER_AREA)
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        break

    frame2 = cv2.resize(frame2, dim, interpolation = cv2.INTER_AREA)
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    Mags . append(mag.ravel())
    Angs.append(ang.ravel())
    
    prvs = next
Mags = np.array(Mags)
Angs = np.array(Angs)
feature_vec = np.append(Mags,Angs,axis=1)
feature_vec.shape

181 640 826


(180, 384)

In [74]:
bow_testt = []
n_clusters = 220
labels = vq(feature_vec, clusters3)[0]
hist = np.histogram(labels,bins = np.arange(n_clusters+1))[0]
bow_testt.append(hist)
bow_testt.append(hist)
bow_testt = np.array(bow_testt)   
X_test = bow_testt
y_pred = clf3.predict(X_test)
y_pred

array([1., 1.])